# Analysis
Please make sure to have the necessary data folders placed in the same directory where you place this notebook.

In [101]:
from __future__ import annotations

import csv
import html
import json
import random
import re
import urllib.request
from pathlib import Path
from typing import Any, Dict, Iterable, List, Optional, Set, Tuple

import matplotlib.pyplot as plt


In [102]:
# Configuration

COUNT_STRING_TOKENS_TOO = False
PAN_ROOT = Path("pan25-multi-author-analysis")
PAN_SAMPLE_N = 250
RANDOM_SEED = 42
JSON_SAMPLE_MATCH_TXT = False
DEBUG_PAN_DISCOVERY = False
LEXICON_DIR = Path("lexicons_cache")
LEXICON_DIR.mkdir(parents=True, exist_ok=True)

# HurtLex URLs (public)- if you are offline, manually download to lexicons_cache/hurtlex_<lang>.tsv
HURTLEX_LANG_URLS: Dict[str, str] = {
    "en": "https://raw.githubusercontent.com/valeriobasile/hurtlex/master/lexica/EN/1.2/hurtlex_EN.tsv",
    "it": "https://raw.githubusercontent.com/valeriobasile/hurtlex/master/lexica/IT/1.2/hurtlex_IT.tsv",
    "pl": "https://raw.githubusercontent.com/valeriobasile/hurtlex/master/lexica/PL/1.2/hurtlex_PL.tsv",
    "da": "https://raw.githubusercontent.com/valeriobasile/hurtlex/master/lexica/DA/1.2/hurtlex_DA.tsv",
}

HURTLEX_RESTRICT_TO_COMMON_TOX_CATEGORIES = True
COMMON_TOX_CATS = {
    "ps",
    "pa",
    "ddf",
    "asf",
    "asm",
    "isf",
    "ism",
    "pr",
    "or",
    "om",
    "re",
    "sv",
}

HURTLEX_LEVELS_ALLOWED: Optional[Set[str]] = {"conservative"}
_HURTLEX_CACHE: Dict[str, Set[str]] = {}


In [103]:
# Sentence splitting helpers   

HARD_WRAP_MIN_CHARS = 55

def _unescape_basic_html(text: str) -> str:
    t = html.unescape(text)
    t = t.replace("&gt", ">").replace("&lt", "<").replace("&amp", "&")
    return t

def _strip_leading_quotes(line: str) -> str:
    s = line.lstrip()
    prefixes = (">", "&gt;", "&gt")
    changed = True
    while changed:
        changed = False
        for pref in prefixes:
            if s.startswith(pref):
                s = s[len(pref):].lstrip()
                changed = True
    return s

WORD_RE = re.compile(r"[^\W_]+(?:[’'\-][^\W_]+)*", re.UNICODE)

def iter_files(folder: Path, pattern: str, recursive: bool = True) -> Iterable[Path]:
    if not folder.exists():
        return
    if recursive:
        yield from folder.rglob(pattern)
    else:
        yield from folder.glob(pattern)

def iter_json_files(folder: Path, recursive: bool = True) -> Iterable[Path]:
    if not folder.exists():
        return
    it = folder.rglob("*") if recursive else folder.glob("*")
    for p in it:
        if p.is_file() and p.suffix.lower() == ".json":
            yield p

def read_text(path: Path) -> str:
    try:
        return path.read_text(encoding="utf-8")
    except UnicodeDecodeError:
        return path.read_text(encoding="utf-8", errors="replace")

def choose_sample(paths: List[Path], sample_n: Optional[int], rng: random.Random) -> List[Path]:
    if sample_n is None or sample_n <= 0 or sample_n >= len(paths):
        return paths
    return rng.sample(paths, k=sample_n)

def list_txt_files(folder: Path) -> List[Path]:
    return sorted(iter_files(folder, "*.txt", recursive=True))

def split_sentences(text: str) -> List[str]:
    if not text:
        return []
    t = _unescape_basic_html(text)
    t = t.replace("\r\n", "\n").replace("\r", "\n")
    sentences: List[str] = []
    for ln in t.split("\n"):
        ln = _strip_leading_quotes(ln)
        ln = " ".join(ln.split())  # collapse whitespace
        if not ln:
            continue
        sentences.append(ln)
    return sentences

def _tokenize(text: str) -> List[str]:
    return [w.lower() for w in WORD_RE.findall(text)]


In [104]:
# Text level statistics

def compute_stats_for_text(text: str) -> Tuple[int, int, int, Optional[int], Optional[int]]:
    parts = split_sentences(text)
    sentence_count = 0
    word_count = 0
    total_word_chars = 0
    min_sentence_len_words: Optional[int] = None
    max_sentence_len_words: Optional[int] = None
    for sent in parts:
        words = WORD_RE.findall(sent)
        if not words:
            continue
        if len(words) == 1 and words[0].lower() in {"gt", "lt", "amp"}:
            continue
        sentence_count += 1
        sent_len = len(words)
        word_count += sent_len
        total_word_chars += sum(len(w) for w in words)
        if min_sentence_len_words is None or sent_len < min_sentence_len_words:
            min_sentence_len_words = sent_len
        if max_sentence_len_words is None or sent_len > max_sentence_len_words:
            max_sentence_len_words = sent_len
    return sentence_count, word_count, total_word_chars, min_sentence_len_words, max_sentence_len_words

def compute_text_stats_for_paths(txt_paths: List[Path]) -> Dict[str, Optional[float]]:
    sentence_count = 0
    word_count = 0
    total_word_chars = 0
    min_sentence_len_words: Optional[int] = None
    max_sentence_len_words: Optional[int] = None
    min_sentence_len_files: List[str] = []
    max_sentence_len_file: Optional[str] = None
    sentences_per_doc: List[int] = []
    for p in txt_paths:
        text = read_text(p)
        s_cnt, w_cnt, w_chars, doc_min, doc_max = compute_stats_for_text(text)
        sentence_count += s_cnt
        word_count += w_cnt
        total_word_chars += w_chars
        if s_cnt > 0:
            sentences_per_doc.append(s_cnt)
        if doc_min is not None:
            if min_sentence_len_words is None or doc_min < min_sentence_len_words:
                min_sentence_len_words = doc_min
                min_sentence_len_files = [str(p)]
            elif doc_min == min_sentence_len_words:
                if str(p) not in min_sentence_len_files and len(min_sentence_len_files) < 2:
                    min_sentence_len_files.append(str(p))
        if doc_max is not None:
            if max_sentence_len_words is None or doc_max > max_sentence_len_words:
                max_sentence_len_words = doc_max
                max_sentence_len_file = str(p)
    avg_sentence_len = (word_count / sentence_count) if sentence_count > 0 else None
    avg_word_len = (total_word_chars / word_count) if word_count > 0 else None
    min_sentences_per_doc = min(sentences_per_doc) if sentences_per_doc else None
    max_sentences_per_doc = max(sentences_per_doc) if sentences_per_doc else None
    avg_sentences_per_doc = (sum(sentences_per_doc) / len(sentences_per_doc)) if sentences_per_doc else None
    return {
        "txt_files": len(txt_paths),
        "sentences": sentence_count,
        "words": word_count,
        "avg_sentence_len_words": avg_sentence_len,
        "min_sentence_len_words": float(min_sentence_len_words) if min_sentence_len_words is not None else None,
        "min_sentence_len_files": min_sentence_len_files,
        "max_sentence_len_words": float(max_sentence_len_words) if max_sentence_len_words is not None else None,
        "max_sentence_len_file": max_sentence_len_file,
        "avg_word_len_chars": avg_word_len,
        "min_sentences_per_doc": float(min_sentences_per_doc) if min_sentences_per_doc is not None else None,
        "max_sentences_per_doc": float(max_sentences_per_doc) if max_sentences_per_doc is not None else None,
        "avg_sentences_per_doc": avg_sentences_per_doc,
    }


In [105]:
# Informality & register markers:
# The informality score used here is a simplistic heuristic based on a limited subset of markers- mainly orthographic ones. 
# As a result, it provides very incomplete coverage and should be interpreted with caution, just as an additional check.

REPEATED_PUNCT_RE = re.compile(r"([!?])\1{1,}|[!?]{3,}")
ELONGATION_RE = re.compile(r"([A-Za-zÀ-ÖØ-öø-ÿ])\1{2,}", re.UNICODE)
ALL_CAPS_TOKEN_RE = re.compile(r"\b[A-Z]{2,}\b")

def compute_informality_for_paths(txt_paths: List[Path]) -> Dict[str, float]:
    tokens_total = 0
    repeated_punct_total = 0
    all_caps_total = 0
    elong_total = 0
    for p in txt_paths:
        text = read_text(p)
        toks = _tokenize(text)
        if not toks:
            continue
        tokens_total += len(toks)
        repeated_punct_total += len(REPEATED_PUNCT_RE.findall(text))
        all_caps_total += len(ALL_CAPS_TOKEN_RE.findall(text))
        elong_total += sum(1 for t in toks if ELONGATION_RE.search(t))
    if tokens_total == 0:
        return {
            "repeated_punct_rate": 0.0,
            "all_caps_token_rate": 0.0,
            "elongation_rate": 0.0,
            "informality_score": 0.0,
        }
    per_1k = lambda x: 1000.0 * x / tokens_total
    repeated_punct_rate = per_1k(repeated_punct_total)
    all_caps_token_rate = per_1k(all_caps_total)
    elongation_rate = per_1k(elong_total)
    return {
        "repeated_punct_rate": repeated_punct_rate,
        "all_caps_token_rate": all_caps_token_rate,
        "elongation_rate": elongation_rate,
        "informality_score": repeated_punct_rate + all_caps_token_rate + elongation_rate,
    }


In [106]:
# Toxicity detection: this loads HurtLex corpora and computes:
# - toxic tokens total
# - toxic tokens per 1k tokens
# - % documents with ≥1 toxic token

def _download(url: str, dest: Path) -> None:
    dest.parent.mkdir(parents=True, exist_ok=True)
    with urllib.request.urlopen(url) as r:
        dest.write_bytes(r.read())

def load_hurtlex(lang: str) -> Set[str]:
    lang = lang.lower()
    if lang in _HURTLEX_CACHE:
        return _HURTLEX_CACHE[lang]
    cache_path = LEXICON_DIR / f"hurtlex_{lang}.tsv"
    if not cache_path.exists():
        url = HURTLEX_LANG_URLS.get(lang)
        if not url:
            print(f"[WARN] No HurtLex URL configured for lang={lang}; toxicity will be 0.")
            _HURTLEX_CACHE[lang] = set()
            return _HURTLEX_CACHE[lang]
        try:
            print(f"[INFO] Downloading HurtLex {lang} -> {cache_path}")
            _download(url, cache_path)
        except Exception as e:
            print(f"[WARN] Failed to download HurtLex for lang={lang}: {e}. Toxicity will be 0.")
            _HURTLEX_CACHE[lang] = set()
            return _HURTLEX_CACHE[lang]
    entries: Set[str] = set()
    try:
        with cache_path.open("r", encoding="utf-8", errors="replace", newline="") as f:
            reader = csv.DictReader(f, delimiter="\t")
            for row in reader:
                lemma = (row.get("lemma") or row.get("term") or row.get("word") or "").strip().lower()
                cat = (row.get("category") or row.get("cat") or "").strip().lower()

                level = (row.get("level") or "").strip().lower()
                if HURTLEX_LEVELS_ALLOWED and level and level not in HURTLEX_LEVELS_ALLOWED:
                    continue
                if not lemma:
                    continue
                if HURTLEX_RESTRICT_TO_COMMON_TOX_CATEGORIES and cat and cat not in COMMON_TOX_CATS:
                    continue
                entries.add(lemma)
    except Exception as e:
        print(f"[WARN] Failed to parse HurtLex file {cache_path}: {e}. Toxicity will be 0.")
        entries = set()
    _HURTLEX_CACHE[lang] = entries
    return entries

def detect_language_for_folder(folder: Path) -> str:
    name = folder.name.lower()
    if "polish" in name:
        return "pl"
    if "italian" in name:
        return "it"
    if "danish" in name:
        return "da"
    return "en"

def compute_toxicity_for_paths(txt_paths: List[Path], lang: str) -> Dict[str, float]:
    lex = load_hurtlex(lang)
    docs_with_toxic = 0
    tokens_total = 0
    toxic_tokens_total = 0
    for p in txt_paths:
        text = read_text(p)
        toks = _tokenize(text)
        if not toks:
            continue
        tokens_total += len(toks)
        toxic_here = sum(1 for t in toks if t in lex)
        toxic_tokens_total += toxic_here
        if toxic_here > 0:
            docs_with_toxic += 1
    toxic_per_1k = (1000.0 * toxic_tokens_total / tokens_total) if tokens_total else 0.0
    toxic_doc_rate = (100.0 * docs_with_toxic / len(txt_paths)) if txt_paths else 0.0
    return {
        "tox_lang": lang,
        "toxic_tokens": float(toxic_tokens_total),
        "toxic_per_1k": toxic_per_1k,
        "toxic_doc_rate_%": toxic_doc_rate,
    }


In [107]:
# Number of authorship changes: counts `0` and `1` inside JSON structures. By default it counts across all JSON files.

def safe_load_json(path: Path) -> Any:
    try:
        raw = path.read_text(encoding="utf-8")
    except UnicodeDecodeError:
        raw = path.read_text(encoding="utf-8", errors="replace")
    return json.loads(raw)

def count_zero_one_tokens(obj: Any) -> Tuple[int, int]:
    zeros = 0
    ones = 0
    stack = [obj]
    while stack:
        cur = stack.pop()
        if isinstance(cur, dict):
            stack.extend(cur.values())
        elif isinstance(cur, list):
            stack.extend(cur)
        elif isinstance(cur, (int, float)):
            if isinstance(cur, bool):
                continue
            if cur == 0:
                zeros += 1
            elif cur == 1:
                ones += 1
        elif COUNT_STRING_TOKENS_TOO and isinstance(cur, str):
            if cur == "0":
                zeros += 1
            elif cur == "1":
                ones += 1
    return zeros, ones

def compute_json_counts_for_folder(
    folder: Path, recursive: bool = True, allowed_stems: Optional[Set[str]] = None
) -> Dict[str, int]:
    json_files = 0
    zeros_total = 0
    ones_total = 0
    parse_errors = 0
    for jpath in iter_json_files(folder, recursive=recursive):
        if allowed_stems is not None and jpath.stem not in allowed_stems:
            continue
        json_files += 1
        try:
            obj = safe_load_json(jpath)
        except Exception:
            parse_errors += 1
            continue
        z, o = count_zero_one_tokens(obj)
        zeros_total += z
        ones_total += o
    return {
        "json_files": json_files,
        "json_parse_errors": parse_errors,
        "zeros": zeros_total,
        "ones": ones_total,
    }


In [108]:
# Plots and table reports

def pct(z: int, o: int) -> Tuple[float, float]:
    total = z + o
    if total == 0:
        return 0.0, 0.0
    return (100.0 * z / total), (100.0 * o / total)

def print_report(results: List[Dict[str, Any]]) -> None:
    headers = [
        "folder",
        "txt_files",
        "avg_sentence_len_words",
        "min_sentence_len_words",
        "max_sentence_len_words",
        "avg_word_len_chars",
        "min_sentences_per_doc",
        "max_sentences_per_doc",
        "avg_sentences_per_doc",
        "json_files",
        "json_parse_errors",
        "zeros",
        "ones",
        "zeros_%",
        "ones_%",
        "repeated_punct_rate",
        "all_caps_token_rate",
        "elongation_rate",
        "informality_score",
        "tox_lang",
        "toxic_tokens",
        "toxic_per_1k",
        "toxic_doc_rate_%",
        "sampled_txt",
    ]
    def fmt(v: Any) -> str:
        if v is None:
            return "NA"
        if isinstance(v, float):
            return f"{v:.3f}"
        return str(v)
    rows = [[fmt(r.get(h)) for h in headers] for r in results]
    widths = [max(len(h), *(len(row[i]) for row in rows)) for i, h in enumerate(headers)]
    def line(cols):
        return " | ".join(c.ljust(widths[i]) for i, c in enumerate(cols))
    print(line(headers))
    print("-+-".join("-" * w for w in widths))
    for row in rows:
        print(line(row))

def _filtered_for_plot(results: List[Dict[str, Any]], key: str) -> List[Dict[str, Any]]:
    return [r for r in results if float(r.get(key, 0.0) or 0.0) != 0.0]

def _distinct_bar_colors(n: int):
    cmap = plt.get_cmap("tab20")
    return [cmap(i % cmap.N) for i in range(n)]

def plot_metric_bar(
    results: List[Dict[str, Any]],
    outdir: Path,
    key: str,
    title: str,
    ylabel: str,
    filename: str,
) -> None:
    data = _filtered_for_plot(results, key)
    if not data:
        print(f"[WARN] No non-zero '{key}' values found; skipping {filename}.")
        return
    labels = [r["folder_name"] for r in data]
    values = [float(r.get(key, 0.0) or 0.0) for r in data]
    x = range(len(labels))
    plt.figure(figsize=(max(10, len(labels) * 1.2), 6))
    plt.bar(list(x), values, color=_distinct_bar_colors(len(values)))
    plt.xticks(list(x), labels, rotation=30, ha="right")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.tight_layout()
    plt.savefig(outdir / filename, dpi=200)
    plt.close()

def plot_json_01(results: List[Dict[str, Any]], outdir: Path) -> None:
    data = [r for r in results if (r.get("zeros", 0) + r.get("ones", 0)) > 0]
    if not data:
        print("[WARN] No JSON 0/1 tokens found; skipping JSON plots.")
        return
    labels = [r["folder_name"] for r in data]
    x = range(len(labels))
    zeros_pct, ones_pct = [], []
    for r in data:
        zp, op = pct(r["zeros"], r["ones"])
        zeros_pct.append(zp)
        ones_pct.append(op)
    plt.figure(figsize=(max(10, len(labels) * 1.2), 6))
    plt.bar([i - 0.2 for i in x], zeros_pct, width=0.4, label="0 %")
    plt.bar([i + 0.2 for i in x], ones_pct, width=0.4, label="1 %")
    plt.xticks(list(x), labels, rotation=30, ha="right")
    plt.ylabel("Percentage (%)")
    plt.ylim(0, 100)
    plt.title("Percentages of 0 vs 1 in the corpora (per folder)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(outdir / "json_0_vs_1_percentages_per_folder.png", dpi=200)
    plt.close()

def plot_hurtlex_lexicon_sizes(lex_sizes: Dict[str, int], outdir: Path) -> None:
    if not lex_sizes:
        print("[WARN] No HurtLex lexicon sizes provided; skipping HurtLex size plot.")
        return
    order = ["en", "it", "pl", "da"]
    langs = [l for l in order if l in lex_sizes] + [l for l in lex_sizes.keys() if l not in order]
    values = [int(lex_sizes[l]) for l in langs]
    plt.figure(figsize=(8, 5))
    plt.bar(langs, values, color=_distinct_bar_colors(len(values)))
    plt.xlabel("Language")
    plt.ylabel("Unique lemmas")
    plt.title("HurtLex lexicon sizes (after filters)")
    for i, v in enumerate(values):
        plt.text(i, v, str(v), ha="center", va="bottom")
    plt.tight_layout()
    plt.savefig(outdir / "hurtlex_lexicon_sizes_after_filters.png", dpi=200)
    plt.close()


In [109]:
# we used this to extract example sentences of specific lengths for error analysis, and a sanity check

def sentences_with_word_len_in_file(path: Path, target_len: int, max_examples: int = 25) -> List[str]:
    try:
        text = read_text(path)
    except Exception:
        return []
    out: List[str] = []
    for sent in split_sentences(text):
        words = WORD_RE.findall(sent)
        if not words:
            continue
        if len(words) == 1 and words[0].lower() in {"gt", "lt", "amp"}:
            continue
        if len(words) == target_len:
            out.append(sent)
            if len(out) >= max_examples:
                break
    return out


In [110]:
# run info on size of HurtLex corpora after filters

rng = random.Random(RANDOM_SEED)
outdir = Path("plots_out")
outdir.mkdir(parents=True, exist_ok=True)

# HurtLex lexicon sizes after filters
hurtlex_sizes: Dict[str, int] = {}
print("\n=== HurtLex lexicon sizes (after filters) ===")
for lang in ["en", "it", "pl", "da"]:
    lex = load_hurtlex(lang)
    hurtlex_sizes[lang] = len(lex)
    print(f"{lang}: {len(lex)} unique lemmas")
print("===========================================\n")

plot_hurtlex_lexicon_sizes(hurtlex_sizes, outdir)



=== HurtLex lexicon sizes (after filters) ===
en: 700 unique lemmas
it: 561 unique lemmas
pl: 412 unique lemmas
da: 242 unique lemmas



In [111]:
# run report

reddit_folders = [
    Path("reddit_dataset_new_POLISH"),
    Path("reddit_dataset_new_ITALIAN"),
    Path("reddit_dataset_new_DANISH"),
]

pan_folders = [
    PAN_ROOT / "easy" / "train",
    PAN_ROOT / "easy" / "validation",
    PAN_ROOT / "medium" / "train",
    PAN_ROOT / "medium" / "validation",
    PAN_ROOT / "hard" / "train",
    PAN_ROOT / "hard" / "validation",
]

all_folders = reddit_folders + pan_folders
results: List[Dict[str, Any]] = []

for folder in all_folders:
    if folder.name in {"train", "validation"}:
        folder_name = f"{folder.parent.name}/{folder.name}"
    else:
        folder_name = folder.name if folder.name else str(folder)
    row: Dict[str, Any] = {"folder": str(folder), "folder_name": folder_name}
    if not folder.exists():
        row.update(
            {
                "txt_files": 0,
                "sentences": 0,
                "words": 0,
                "avg_sentence_len_words": None,
                "min_sentence_len_words": None,
                "max_sentence_len_words": None,
                "avg_word_len_chars": None,
                "min_sentences_per_doc": None,
                "max_sentences_per_doc": None,
                "avg_sentences_per_doc": None,
                "json_files": 0,
                "json_parse_errors": 0,
                "zeros": 0,
                "ones": 0,
                "zeros_%": 0.0,
                "ones_%": 0.0,
                "repeated_punct_rate": 0.0,
                "all_caps_token_rate": 0.0,
                "elongation_rate": 0.0,
                "informality_score": 0.0,
                "tox_lang": detect_language_for_folder(folder),
                "toxic_tokens": 0.0,
                "toxic_per_1k": 0.0,
                "toxic_doc_rate_%": 0.0,
                "sampled_txt": "NA",
            }
        )
        results.append(row)
        continue
    is_pan_split = PAN_ROOT in folder.parents
    sample_n = PAN_SAMPLE_N if is_pan_split else None  # Reddit uses ALL docs
    all_txt = list_txt_files(folder)
    sampled_txt = choose_sample(all_txt, sample_n=sample_n, rng=rng)
    row["sampled_txt"] = f"{len(sampled_txt)}/{len(all_txt)}" if all_txt else "0/0"
    if DEBUG_PAN_DISCOVERY and is_pan_split:
        sample_json = next(iter(iter_json_files(folder, recursive=True)), None)
        if sample_json is not None:
            print(f"[OK] Found JSON under {folder}: e.g. {sample_json.name}")
        else:
            print(f"[WARN] No JSON found under {folder} (check root/path/nesting)")
        print(f"[INFO] PAN sampling for {folder_name}: {row['sampled_txt']} (seed={RANDOM_SEED})")
    row.update(compute_text_stats_for_paths(sampled_txt))
    row.update(compute_informality_for_paths(sampled_txt))
    lang = detect_language_for_folder(folder)
    row.update(compute_toxicity_for_paths(sampled_txt, lang=lang))
    # JSON authorship changes
    allowed_stems = {p.stem for p in sampled_txt} if (JSON_SAMPLE_MATCH_TXT and sampled_txt) else None
    row.update(compute_json_counts_for_folder(folder, recursive=True, allowed_stems=allowed_stems))
    zp, op = pct(row["zeros"], row["ones"])
    row["zeros_%"], row["ones_%"] = zp, op
    results.append(row)

print_report(results)


folder                                        | txt_files | avg_sentence_len_words | min_sentence_len_words | max_sentence_len_words | avg_word_len_chars | min_sentences_per_doc | max_sentences_per_doc | avg_sentences_per_doc | json_files | json_parse_errors | zeros | ones  | zeros_% | ones_% | repeated_punct_rate | all_caps_token_rate | elongation_rate | informality_score | tox_lang | toxic_tokens | toxic_per_1k | toxic_doc_rate_% | sampled_txt
----------------------------------------------+-----------+------------------------+------------------------+------------------------+--------------------+-----------------------+-----------------------+-----------------------+------------+-------------------+-------+-------+---------+--------+---------------------+---------------------+-----------------+-------------------+----------+--------------+--------------+------------------+------------
reddit_dataset_new_POLISH                     | 250       | 13.102                 | 2.000          

In [112]:
# inspecting selected folders to see example sentences of min lengths

PRINT_INSPECT_FOLDERS = {
    str(Path("reddit_dataset_new_POLISH")),
    str(Path("reddit_dataset_new_ITALIAN")),
    str(Path("reddit_dataset_new_DANISH")),
    str(PAN_ROOT / "hard" / "validation"),
}

for r in results:
    folder_path = str(r.get("folder", ""))
    if folder_path not in PRINT_INSPECT_FOLDERS:
        continue
    max_fname = r.get("max_sentence_len_file")
    print(f"[MAX_SENT_LEN_FILE] {folder_path}: {Path(max_fname).name if max_fname else 'NA'}")
    min_files = r.get("min_sentence_len_files") or []
    if not isinstance(min_files, list):
        min_files = [str(min_files)]
    min_files = [str(x) for x in min_files if x][:2]
    print(f"[MIN_SENT_LEN_FILES] {folder_path}: {', '.join(Path(x).name for x in min_files) if min_files else 'NA'}")
    try:
        min_len = int(float(r.get("min_sentence_len_words"))) if r.get("min_sentence_len_words") is not None else None
    except Exception:
        min_len = None
    if min_len is None or not min_files:
        print(f"[MIN_SENT_LEN_SENTENCES] {folder_path}: NA")
        continue
    for fp in min_files:
        p = Path(fp)
        examples = sentences_with_word_len_in_file(p, min_len, max_examples=25)
        print(f"[MIN_SENT_LEN_SENTENCES] {folder_path}: {p.name} (len={min_len})")
        if not examples:
            print("  - NA")
        else:
            for s in examples:
                print(f"  - {s}")


[MAX_SENT_LEN_FILE] reddit_dataset_new_POLISH: problem-239.txt
[MIN_SENT_LEN_FILES] reddit_dataset_new_POLISH: problem-150.txt, problem-234.txt
[MIN_SENT_LEN_SENTENCES] reddit_dataset_new_POLISH: problem-150.txt (len=2)
  - tyle wygrać 🏆 👏🏻
[MIN_SENT_LEN_SENTENCES] reddit_dataset_new_POLISH: problem-234.txt (len=2)
  - Do zobaczenia!
[MAX_SENT_LEN_FILE] reddit_dataset_new_ITALIAN: problem-219.txt
[MIN_SENT_LEN_FILES] reddit_dataset_new_ITALIAN: problem-100.txt, problem-116.txt
[MIN_SENT_LEN_SENTENCES] reddit_dataset_new_ITALIAN: problem-100.txt (len=3)
  - I “bi” locali 😭😭😭
[MIN_SENT_LEN_SENTENCES] reddit_dataset_new_ITALIAN: problem-116.txt (len=3)
  - Tagli le ruote 🔪
[MAX_SENT_LEN_FILE] reddit_dataset_new_DANISH: problem-208.txt
[MIN_SENT_LEN_FILES] reddit_dataset_new_DANISH: problem-103.txt, problem-127.txt
[MIN_SENT_LEN_SENTENCES] reddit_dataset_new_DANISH: problem-103.txt (len=3)
  - Rigtig flot gjort 😎😎👍
[MIN_SENT_LEN_SENTENCES] reddit_dataset_new_DANISH: problem-127.txt (len=3)

In [113]:
# plots

plot_json_01(results, outdir)

plot_metric_bar(
    results,
    outdir,
    key="informality_score",
    title="Informality score (per 1k tokens) by folder",
    ylabel="Informality score (per 1k tokens)",
    filename="informality_score_per_folder.png",
)

plot_metric_bar(
    results,
    outdir,
    key="toxic_per_1k",
    title="Toxic word rate (HurtLex) per 1k tokens by folder",
    ylabel="Toxic words per 1k tokens",
    filename="toxic_words_per_1k_per_folder.png",
)

print(f"Saved the following plots:")
print(" - json_0_vs_1_percentages_per_folder.png")
print(" - informality_score_per_folder.png")
print(" - toxic_words_per_1k_per_folder.png")
print(" - hurtlex_lexicon_sizes_after_filters.png")


Saved the following plots:
 - json_0_vs_1_percentages_per_folder.png
 - informality_score_per_folder.png
 - toxic_words_per_1k_per_folder.png
 - hurtlex_lexicon_sizes_after_filters.png


### Other analyses

In [114]:
# here we normalize the toxic word rates by HurtLex lexicon sizes

try:
    hurtlex_sizes
except NameError:
    hurtlex_sizes = {}
    for _lang in ["en", "it", "pl", "da"]:
        hurtlex_sizes[_lang] = len(load_hurtlex(_lang))

_sizes = [v for v in hurtlex_sizes.values() if isinstance(v, int) and v > 0]
if not _sizes:
    raise ValueError("No HurtLex sizes available--- cannot normalize.")
_sizes_sorted = sorted(_sizes)
_mid = len(_sizes_sorted) // 2
ref_size = float(_sizes_sorted[_mid]) if (len(_sizes_sorted) % 2 == 1) else float((_sizes_sorted[_mid - 1] + _sizes_sorted[_mid]) / 2)

print("HurtLex size normalization")
print(f"Reference size (median): {ref_size:.1f} lemmas")
for _lang, _sz in sorted(hurtlex_sizes.items()):
    if _sz > 0:
        print(f"  {_lang}: {_sz} lemmas | factor = {ref_size / _sz:.4f}")
    else:
        print(f"  {_lang}: {_sz} lemmas | factor = NA")
print("==========================\n")

for r in results:
    lang = (r.get("tox_lang") or "en").lower()
    lex_sz = hurtlex_sizes.get(lang, 0)
    tox = float(r.get("toxic_per_1k") or 0.0)
    factor = (ref_size / lex_sz) if lex_sz else 1.0
    r["toxic_per_1k_lexnorm"] = tox * factor

plot_metric_bar(
    results,
    outdir,
    key="toxic_per_1k_lexnorm",
    title="Toxic word rate (HurtLex) per 1k tokens by folder (lexicon-size normalized)",
    ylabel="Toxic words per 1k tokens (lexicon-size normalized)",
    filename="toxic_words_per_1k_per_folder_normalized.png",
)

print("Saved plot:")
print("toxic_words_per_1k_per_folder_normalized.png")

HurtLex size normalization
Reference size (median): 486.5 lemmas
  da: 242 lemmas | factor = 2.0103
  en: 700 lemmas | factor = 0.6950
  it: 561 lemmas | factor = 0.8672
  pl: 412 lemmas | factor = 1.1808

Saved plot:
toxic_words_per_1k_per_folder_normalized.png


In [115]:
# analyzing and detecting the amount of bot-generated phrases or community rules that repeat in the datasets
# doing this only for PAN datasets here

TARGET_BOT_PHRASES = (
    "Debate/discuss/argue the merits of ideas, don't attack people.",
    "I am a bot, and this action was performed automatically.",
)

PAN_FOLDERS = [
    PAN_ROOT / "easy" / "train",
    PAN_ROOT / "easy" / "validation",
    PAN_ROOT / "medium" / "train",
    PAN_ROOT / "medium" / "validation",
    PAN_ROOT / "hard" / "train",
    PAN_ROOT / "hard" / "validation",
]

def read_text_utf8(path: Path) -> str:
    try:
        return path.read_text(encoding="utf-8")
    except UnicodeDecodeError:
        return path.read_text(encoding="utf-8", errors="replace")

def find_matching_txt_files(folder: Path, phrases: tuple[str, ...]) -> list[Path]:
    if not folder.exists():
        return []
    matched: list[Path] = []
    for path in folder.rglob("*.txt"):
        if not path.is_file():
            continue
        text = read_text_utf8(path)
        if any(phrase in text for phrase in phrases):
            matched.append(path)
    return sorted(matched)

print("Results in PAN folders")
print("Target phrases:")
for phrase in TARGET_BOT_PHRASES:
    print(f" - {phrase}")
print()

for folder in PAN_FOLDERS:
    folder_disp = f"{folder.parent.name}/{folder.name}"  # e.g., easy/train
    matched = find_matching_txt_files(folder, TARGET_BOT_PHRASES)
    print(f"{folder_disp}: {len(matched)} matched documents")
    # for path in matched:
    #    print(f"  - {path.name}")


Results in PAN folders
Target phrases:
 - Debate/discuss/argue the merits of ideas, don't attack people.
 - I am a bot, and this action was performed automatically.

easy/train: 563 matched documents
easy/validation: 133 matched documents
medium/train: 2305 matched documents
medium/validation: 510 matched documents
hard/train: 13 matched documents
hard/validation: 2 matched documents
